<a href="https://colab.research.google.com/github/detkartik/upgrad_assignment/blob/master/sentiment_based_recommendation_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load the data
data = pd.read_csv('/content/drive/MyDrive/sample30 (1).csv')

# Drop rows with missing values in 'reviews_text' and 'user_sentiment'
data = data.dropna(subset=['reviews_text', 'user_sentiment'])

# Clean text data
data['reviews_text'] = data['reviews_text'].str.lower().str.replace('[^\w\s]', '')

# Encode user sentiment
data['user_sentiment'] = data['user_sentiment'].map({'Positive': 1, 'Negative': 0})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['reviews_text'], data['user_sentiment'], test_size=0.2, random_state=42)

# Vectorize text data
vectorizer = CountVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train the model
model = LogisticRegression()
model.fit(X_train_vec, y_train)

# Predict on the test set
y_pred = model.predict(X_test_vec)

# Evaluate the model
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


<ipython-input-1-23eead3fa912>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['reviews_text'] = data['reviews_text'].str.lower().str.replace('[^\w\s]', '')
<ipython-input-1-23eead3fa912>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['user_sentiment'] = data['user_sentiment'].map({'Positive': 1, 'Negative': 0})


Accuracy: 0.9448333333333333
              precision    recall  f1-score   support

           0       0.79      0.67      0.73       653
           1       0.96      0.98      0.97      5347

    accuracy                           0.94      6000
   macro avg       0.88      0.83      0.85      6000
weighted avg       0.94      0.94      0.94      6000



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [2]:
!pip install scikit-surprise

In [3]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(1, 5))

# Load the data from the DataFrame
data_surprise = Dataset.load_from_df(data[['reviews_username', 'name', 'reviews_rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data_surprise, test_size=0.25)

# Use the SVD algorithm
algo = SVD()

# Train the algorithm on the trainset
algo.fit(trainset)

# Test the algorithm on the testset
predictions = algo.test(testset)

# Compute and print the RMSE
print('RMSE:', accuracy.rmse(predictions))

RMSE: 0.7743
RMSE: 0.7742738926506441


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Combine relevant features for content-based filtering
data['combined_features'] = data['name'] + " " + data['categories'] + " " + data['reviews_text']

# Vectorize the combined features
vectorizer = TfidfVectorizer(stop_words='english')
content_vectors = vectorizer.fit_transform(data['combined_features'])

# # Compute cosine similarity matrix
batch_size = 1000
num_rows = content_vectors.shape[0]
cosine_sim = np.zeros((num_rows, num_rows))  # Initialize cosine similarity matrix

# Compute cosine similarity in batches
for i in range(0, num_rows, batch_size):
    start_idx = i
    end_idx = min(i + batch_size, num_rows)
    batch_similarity = cosine_similarity(content_vectors[start_idx:end_idx], content_vectors)
    cosine_sim[start_idx:end_idx] = batch_similarity


In [5]:
def get_content_recommendations(product_id, cosine_sim=cosine_sim):
    try:
        idx = data.index[data['id'] == product_id][0]
    except IndexError:
        print(f"Product ID '{product_id}' not found in the dataset.")
        return None

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Get top 5 similar products
    product_indices = [i[0] for i in sim_scores]
    return data['name'].iloc[product_indices]

# Example usage
print(get_content_recommendations('1'))

Product ID '1' not found in the dataset.
None


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/recommend-products', methods=['POST'])
def recommend_products():
    data = request.json
    product_id = data['product_id']

    # Call the recommendation function
    recommendations = get_content_recommendations(product_id)

    if recommendations:
        return jsonify({'recommendations': recommendations})
    else:
        return jsonify({'error': f"Product ID '{product_id}' not found."}), 404

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
